In [4]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import datetime
from IPython.display import clear_output
import re
import csv 

In [5]:
main_url= "https://www.migros.com.tr"
subcat_urls = []
item_urls = []
cat_name = []
all_pages_urls = []

product = {
    "name":[],
    "price":[],
    "producer":[],
    "cat":[]
}

In [6]:
html = requests.get(main_url).content
soup = bs(html, "lxml")

In [7]:
tags = soup.findAll("li", class_="category-list-item category-title")
for item in tags[:81]:
    for i in item.find_all("a", href=True):
        subcat_urls.append(main_url+i["href"])
        cat_name.append(i.text.strip())

In [8]:
#Defining a function to find last page of every catagory
def find_last_page(page):
    r = requests.get(page).content
    soup = bs(r, "lxml")
    a = soup.find("ul", class_="pagination").text.strip()
    temp = re.findall(r'\d+', a)
    res = list(map(int, temp))
    return int(res[-1])

In [ ]:
#Taking the all urls of all pages from website 
for page in subcat_urls:
    last_page = find_last_page(page)
    for i in range(1, last_page+1):
        try:
            new_url = page + str("?sayfa=") + str(i)
            html = requests.get(new_url).text
            soup = bs(html, "lxml")
            all_pages_urls.append(new_url)
        except IndexError:
            break

In [ ]:
#Taking all the items URLs from pages
page_number = 0
for url in all_pages_urls:
    try:
        
        html = requests.get(url).content
        soup = bs(html, "lxml")
        tag = soup.findAll("h5", class_="title product-card-title")
        for item in tag:
            for i in item.find_all("a", href=True):
                item_urls.append(main_url+i["href"])
                page_number += 1
    except IndexError:
            break
print("Number of the webpage: ", page_number)

In [ ]:
#Taking price, name, catogory and the brand from the item pages
i = 0
for link in item_urls:
    try:
        i += 1
        r = requests.get(link).content
        soups = bs(r, "lxml")
        name = soups.find("h1", class_="seo title").text.strip()
        price = soups.find("span", class_="value").text.strip()
        producer = soups.find("span", "sup-title").text.strip()[7:]
        category = soups.find("ul", class_="breadcrumb").text.strip()
        category = category.split("\n")[4].split(" "*12)[1]
        product["name"].append(name)
        product["price"].append(price)
        product["producer"].append(producer)
        product["cat"].append(category)
    except IndexError:
        print(link, ": is passed")

In [ ]:
df = pd.DataFrame(product)

In [ ]:
df.rename(columns = {'name' : 'Ürün ismi', 'price' : 'Fiyat', "producer": "Satıcı", "cat":"Kategori"}, inplace = True)

In [ ]:
path = r'C:\Users\armag\Desktop\Web\Products.csv'
df.to_csv(path, index=False, encoding="utf-8", sep=";")

In [510]:
df.head()

,Ürün ismi,Fiyat,Satıcı,Kategori
0,Muz Yerli Kg,"12,99 TL",Reyondan,Meyve
1,Limon Kg,"7,95 TL",Reyondan,Meyve
2,Muz İthal Kg,"17,95 TL",Reyondan,Meyve
3,Portakal Sıkma File Kg,"6,59 TL",Reyondan,Meyve
4,Avokado Adet,"6,95 TL",Reyondan,Meyve
